<a href="https://colab.research.google.com/github/AlexanderMaslikhin/parsing/blob/homework3/parsing_homework3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [55]:
from bs4 import BeautifulSoup as bs
import requests
import urllib.parse as up
from pprint import pprint
import re
import json

In [59]:
def get_vacancies_hh(vacancy, pages_count=1):
  vlist = []
  headers = {
      "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36"
  }
  for p in range(pages_count):
    if not p:
        url = f"https://hh.ru/search/vacancy?text={up.quote_plus(vacancy, safe='')}&from=suggest_post&area=1"
    else:
        url = f"https://hh.ru/search/vacancy?text={up.quote_plus(vacancy, safe='')}&from=suggest_post&area=1&page={p}&hhtmFrom=vacancy_search_list"
    response = requests.get(url, headers=headers)
    dom = bs(response.text, "html.parser")
    v_list_on_page = dom.select("div.vacancy-serp-item-body")

    for vacansy_block in v_list_on_page:
        v_dict = {}
        a_header = vacansy_block.select("a.serp-item__title")
        v_dict['name'] = a_header[0].text
        v_dict['link'] = a_header[0]["href"]
        salary_block = vacansy_block.select("span.bloko-header-section-3")

        if salary_block:
          salary = {}
          salary_text = re.sub(r'\s', '', salary_block[0].text)
          if salary_text.startswith('от'):
            m = re.search(r'\d+', salary_text)
            salary['from'] = int(m.group(0))
            salary['currency'] = re.sub(r'\d+', '', salary_text[2:])
          elif salary_text.startswith('до'):
            m = re.search(r'\d+', salary_text)
            salary['to'] = int(m.group(0))
            salary['currency'] = re.sub(r'\d+', '', salary_text[2:])
          else:
            salary['from'], salary['to'] = map(int, re.split(r'\D+', salary_text)[:-1])
            salary['currency'] = re.sub(r'\d+–\d+', '', salary_text)
          v_dict['salary'] = salary
          v_dict['employer'] = vacansy_block.select('a.bloko-link.bloko-link_kind-tertiary')[0].text
          v_dict['place'] = vacansy_block.find('div', attrs={"data-qa": "vacancy-serp__vacancy-address"}).text

        vlist.append(v_dict)

  return vlist    



vacancies = get_vacancies_hh("Python разработчик", pages_count=5)
with open("vacancies.json", "w") as f:
  json.dump(vacancies, f, ensure_ascii=False, indent=4)

pprint(vacancies)

[{'employer': 'MANGO FZCO',
  'link': 'https://hh.ru/vacancy/77147428?from=vacancy_search_list&query=Python+%D1%80%D0%B0%D0%B7%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D1%87%D0%B8%D0%BA',
  'name': 'Backend программист (Python)',
  'place': 'Москва',
  'salary': {'currency': 'руб.', 'from': 120000}},
 {'link': 'https://hh.ru/vacancy/76968960?from=vacancy_search_list&query=Python+%D1%80%D0%B0%D0%B7%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D1%87%D0%B8%D0%BA',
  'name': 'Python Developer (проект "Маркетинг ") Middle'},
 {'link': 'https://hh.ru/vacancy/76978400?from=vacancy_search_list&query=Python+%D1%80%D0%B0%D0%B7%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D1%87%D0%B8%D0%BA',
  'name': 'Python Developer (проект "Маркетинг БКП") Middle'},
 {'link': 'https://hh.ru/vacancy/76976985?from=vacancy_search_list&query=Python+%D1%80%D0%B0%D0%B7%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D1%87%D0%B8%D0%BA',
  'name': 'Python Developer (проект "Маркетинг") Senior'},
 {'link': 'https://hh.ru/vacancy/76544925?from=vacancy_search_list&query=Python+